In [ ]:
# Detect current folder to avoid package import error

import os, sys
currentdir = os.path.dirname(os.path.realpath(''))
parentdir = os.path.dirname(currentdir)
sys.path.append(currentdir)

In [ ]:
# Import Opensource packages

import numpy as np
import pandas as pd

import idaes
import pyomo.environ as pyo

from pyomo.core.util import quicksum 
from pyomo.core.expr import evaluate_expression

## Create Concrete Model

In [ ]:
from pyomo.environ import *
# from mydata import * --> import data from separate file

model = ConcreteModel(name = "Price Optimization")

In [ ]:
# Define dictionary

Data_dict = {
    'sites': ['NO', 'DE'],
    'time_periods': ['Q4'],
    
    #'cost_total': {('NO', 'Q4'): 1.32},
    # total cost: (cost_energy * d + cost_revshare * d + cost_fixelec + cost_demand + cost_leasing + cost_sm)
    
    # Variable cost
    
    'cost_energy': {
        ('NO', 'Q4'): 0.21,
        ('DE', 'Q4'): 0.31,
        },
    
    'cost_revshare': {
        ('NO', 'Q4'): 0.05,
        ('DE', 'Q4'): 0.07,
        },

    # Fixed cost
    
    'cost_fixelec': {
        ('NO', 'Q4'): 0.1, 
        ('DE', 'Q4'): 0.12,
        },
    
    'cost_demand': {
        ('NO', 'Q4'): 0.03,
        ('DE', 'Q4'): 0.05,
        },
    
    'cost_leasing': {
        ('NO', 'Q4'): 0.02,
        ('DE', 'Q4'): 0.04,
        },
    
    'cost_sm': {
        ('NO', 'Q4'): 0.05,
        ('DE', 'Q4'): 0.07,
        },
    
    'p_min': 0.1,
    'p_max': 0.9,
    'd_max': 100,
    'd_0_DE': 1.73,
    'd_p_DE': 1.97,
    'd_0_NO': 1.02,
    'd_p_NO': 1.25,
}

# Define sets

model.S = Set(initialize=Data_dict['sites'])
model.T = Set(initialize=Data_dict['time_periods'])

In [ ]:
model.p = Var(
    model.S, 
    model.T,  
    bounds=(Data_dict['p_min'], Data_dict['p_max']),
    domain=NonNegativeReals,
    doc = 'dv: price'
)

### Define Parameters

In [ ]:
model.p_min = Param(initialize=Data_dict['p_min'])
model.p_max = Param(initialize=Data_dict['p_max'])
model.d_max = Param(initialize=Data_dict['d_max'])
model.d_0_DE = Param(Data_dict['d_0_DE'], initialize=Data_dict['d_0_DE'], within=NonNegativeReals)
model.d_p_DE = Param(Data_dict['d_p_DE'], initialize=Data_dict['d_p_DE'], within=NonNegativeReals)
model.d_0_NO = Param(Data_dict['d_0_NO'], initialize=Data_dict['d_0_NO'], within=NonNegativeReals)
model.d_p_NO = Param(Data_dict['d_p_NO'], initialize=Data_dict['d_p_NO'], within=NonNegativeReals)

# Define the demand function for DE and NO based on the price --> e.g. d(p) = 1 - p

def demand_rule(model, s, t):
    if s == 'NO':
        return model.d_0_NO - model.d_p_NO * model.p[s,t]
    elif s == 'DE':
        return model.d_0_DE - model.d_p_DE * model.p[s,t]
model.d = Expression(model.S, model.T, rule=demand_rule)

# Define the total cost calculation for DE and NO

def total_cost_rule(model, s, t):
    if s == 'NO':
        return (
            Data_dict['cost_energy'][s,t] * model.d[s,t] +
            Data_dict['cost_revshare'][s,t] * model.d[s,t] +
            Data_dict['cost_fixelec'][s,t] +
            Data_dict['cost_demand'][s,t] +
            Data_dict['cost_leasing'][s,t] +
            Data_dict['cost_sm'][s,t]
        )
    elif s == 'DE':
        return (
            Data_dict['cost_energy'][s,t] * model.d[s,t] +
            Data_dict['cost_revshare'][s,t] * model.d[s,t] +
            Data_dict['cost_fixelec'][s,t] +
            Data_dict['cost_demand'][s,t] +
            Data_dict['cost_leasing'][s,t] +
            Data_dict['cost_sm'][s,t]
        )
model.c = Expression(model.S, model.T, rule=total_cost_rule)

In [ ]:
# Define objective for DE and NO 

def obj_rule_DE(model):
    return sum(
        model.d[s,t] * (model.p[s,t] - model.c[s,t])
        for s in model.S if s == 'DE' for t in model.T
    )
model.profit_DE = Objective(rule=obj_rule_DE, sense=maximize)

def obj_rule_NO(model):
    return sum(
        model.d[s,t] * (model.p[s,t] - model.c[s,t])
        for s in model.S if s == 'NO' for t in model.T
    )
model.profit_NO = Objective(rule=obj_rule_NO, sense=maximize)

# Alternative

# model.profit = Objective(
    #expr = sum(
        #model.d[s,t]*(model.p[s,t]-model.c[s,t]) \
            #for s in model.S for t in model.T
        #), 
    #sense = maximize
#)

In [ ]:
# Define constraints

def price_constraint_rule(model, s, t):
    return model.p_min <= model.p[s,t] <= model.p_max
model.price_constraint = Constraint(model.S, model.T, rule=price_constraint_rule)

def demand_constraint_rule(model, s, t):
    return model.d[s,t] <= model.d_max
model.demand_constraints = Constraint(model.S, rule=demand_constraint_rule)

In [ ]:
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

# Create a solver instance
solver = SolverFactory('cplex')

# Solve the model
result = solver.solve(model)

# Check the solver status
if result.solver.status == SolverStatus.ok and result.solver.termination_condition == TerminationCondition.optimal:
    # The model was solved to optimality
    print("Optimal solution found.")
    # Access the variable values
    for var in model.component_data_objects(Var):
        print(f"{var.name}: {var.value}")
else:
    # The model failed to solve
    print("Solver did not find an optimal solution.")